<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/151_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 — Kernel Check - Make sure your notebook is running in .venv:
import sys
print("Python executable in use:", sys.executable)


Python executable in use: /Users/micahshull/Documents/AI_Agents/LangChain/LC_setup_day_00/.venv/bin/python3


In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load your API keys
load_dotenv("API_KEYS.env")

# Initialize model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Test it
response = llm.invoke("Say hello from LangChain in one sentence.")
print(response.content)


Hello from LangChain!


In [ ]:
from langchain.prompts import ChatPromptTemplate

# Build a prompt template with a placeholder
prompt = ChatPromptTemplate.from_template("Translate the following English text into French:\n\n{text}")

# Combine prompt + model into a simple chain
chain = prompt | llm

# Run it
result = chain.invoke({"text": "Good morning, how are you?"})
print(result.content)


Bonjour, comment ça va ?



## 🧠 Why `ChatPromptTemplate` is More Flexible Than Hardcoding

Hardcoding a prompt might look like this:

```python
response = llm.invoke("Translate this: Hello!")
```

Seems fine, right? But the moment your project grows, that starts to cause problems:

| Problem         | Hardcoded Prompt                 | ChatPromptTemplate                          |
| --------------- | -------------------------------- | ------------------------------------------- |
| Reusability     | ❌ Duplicated strings all over    | ✅ One place, used many times                |
| Maintainability | ❌ Can't tweak prompts easily     | ✅ Central control over prompt logic         |
| Dynamic Inputs  | ❌ Need to use f-strings manually | ✅ Built-in placeholders and formatting      |
| Testing         | ❌ Hard to isolate logic          | ✅ Easy to unit test input/output formatting |
| Pipelines       | ❌ Hard to chain inputs           | ✅ Plugs into the LangChain pipeline cleanly |

**Example of hardcoding f-strings:**

```python
llm.invoke(f"Translate this: {text}")
```

What if you want to:

* Switch out `{text}` for `{sentence}`
* Use a different language
* Add system instructions?

With `ChatPromptTemplate`, it’s just:

```python
template = ChatPromptTemplate.from_template("Translate to {language}: {text}")
template.format(language="French", text="Hello")
```

🚀 **TL;DR**: Prompt templates make your system **cleaner**, **more scalable**, and **easier to change** without rewriting code all over.

---

## 🛠️ Options for `ChatPromptTemplate`

Here are the ways to create a prompt template:

### 1. `from_template(...)`

* Takes a **string with `{variables}`**.
* Easiest and most common.

```python
ChatPromptTemplate.from_template("What is the capital of {country}?")
```

---

### 2. `from_messages([...])`

* Useful for multi-message chat (e.g. system + user messages).

```python
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "Translate this: {text}")
])
```

🧠 Good for structured multi-turn inputs or formatting for chat models like OpenAI’s GPT-4.

---

### 3. `from_examples(...)`

* Builds few-shot prompts using example inputs and outputs.

```python
ChatPromptTemplate.from_examples(
    examples=[("Hello", "Bonjour"), ("Goodbye", "Au revoir")],
    example_template="Input: {input}\nOutput: {output}",
    suffix="Now translate: {text}",
    input_variables=["text"]
)
```

This is used for few-shot learning, where you show the model examples of the task before the actual input.

---

## 🪄 The `|` Pipe Operator in LangChain (LCEL)

The `|` operator comes from **LangChain Expression Language (LCEL)**.

### 🔁 Think of it like a data pipeline:

```python
prompt | llm | parser
```

* **`prompt`** generates a string
* **`llm`** uses the string as input, returns a response
* **`parser`** turns that output into structured data (e.g. dict, list)

Each component is an **LCEL runnable**, which means:

* It can be invoked
* It can be composed with others
* It can be serialized, saved, or traced

---

### 🔥 Why the Pipe is Powerful

* **Minimal boilerplate** — no messy function calls
* **Composable** — can reuse or mix-and-match components
* **Traceable** — easy to debug intermediate stages
* **Flexible** — swap in new models, prompts, tools, parsers

---

## 🧪 Bonus: You Can Build Chains like LEGO

```python
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("What's the opposite of '{word}'?")
chain = prompt | llm | StrOutputParser()

result = chain.invoke({"word": "hot"})
print(result)  # → "cold"
```




Think of these as **your essential building blocks** for designing any kind of LLM-powered agent.

---

## 🔍 High-Level Breakdown

### 1. `RunnableLambda`, `RunnableMap`

(from `langchain_core.runnables`)

#### 💡 **Purpose**

These are part of **LCEL (LangChain Expression Language)**, which gives you a way to **compose steps together** using the `|` operator (like pipes in Unix).

* `RunnableLambda`: lets you wrap any Python function so it behaves like a LangChain step.
* `RunnableMap`: lets you run multiple runnables in **parallel**, mapping keys to components. Great for branching pipelines.

#### 🤖 **Why they matter**

They turn *everything* into a composable building block — so you can:

* Swap components in and out (LLMs, tools, filters).
* Reuse logic easily.
* Keep agent chains clean and declarative.

---

### 2. `StrOutputParser`

(from `langchain_core.output_parsers`)

#### 💡 **Purpose**

Parses the **raw output** from the LLM (often a structured message or object) into a plain string.

* It simplifies: `"Chat message" → "text only"`
* Useful at the **end of a chain** to get usable data.

#### 🤖 **Why it matters**

Most LLM calls return structured messages (like `AIMessage(content=...)`). You usually want just the string — this does that cleanly.

---

### 3. `AIMessage`, `HumanMessage`

(from `langchain_core.messages`)

#### 💡 **Purpose**

Used to **manually craft chat history** or LLM input/output messages. They're part of the new **message-based LLM architecture**.

* `HumanMessage`: what the user says
* `AIMessage`: what the model says

#### 🤖 **Why they matter**

* Required if you're manually setting up a conversation (chat history, few-shot examples, etc.).
* Helps if you're simulating messages, fine-tuning prompts, or building chat-based memory.

---

### 4. `tool` decorator

(from `langchain_core.tools`)

#### 💡 **Purpose**

Wraps a regular Python function and makes it a **LangChain-compatible tool**. Adds metadata (like name, description, etc.) automatically.

```python
@tool
def fake_weather(location: str) -> str:
    return "Always sunny"
```

* Adds metadata → tool becomes usable inside agents or chains.
* Auto-generates OpenAI-style function definitions (if needed).

#### 🤖 **Why it matters**

LangChain agents can only use tools defined this way. It’s the easiest path from *normal Python code* → *agent-executable action*.

---

## 🧠 Why You Should Care

All of these together form the **core abstraction layer** in LangChain:

* Flexible composition with `RunnableLambda` and `RunnableMap`
* Clean output handling with `StrOutputParser`
* Native chat formatting with `AIMessage`, `HumanMessage`
* Easy tool integration with `@tool`

They're what make LangChain feel like **LEGO blocks for AI systems**.



In [ ]:
# agent_structure_v1.py

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.tools import tool

from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

# Load API key
load_dotenv("API_KEYS.env")
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY")
)

# -------------------------------
# 🛠️ 1. Define a mock tool
# -------------------------------
@tool
def fake_weather_tool(location: str) -> str:
    """Returns a fake weather report for a given location."""
    return f"The weather in {location} is sunny and 72°F."

# Make it usable in LangChain format
tools = {
    "fake_weather_tool": fake_weather_tool,
}

# -------------------------------
# 🧠 2. Create prompt template
# -------------------------------
prompt = ChatPromptTemplate.from_template("""
You are an assistant that uses tools.

User input: {input}

Decide what to do and respond. If a tool is needed, say so.
""")

# -------------------------------
# 🧱 3. Build the chain
# -------------------------------

# Step 1: Prompt -> LLM
chain = prompt | llm | StrOutputParser()

# -------------------------------
# 🚀 4. Run the agent
# -------------------------------
response = chain.invoke({"input": "What's the weather in Paris?"})

print("LLM response:")
print(response)


## 🧠 What This Code Teaches You

### 🔧 `@tool`

* Converts a basic function into a **LangChain Tool**
* Registers name + docstring as metadata
* You’ll later pass these to agents or tool routers

### 📦 `RunnableMap`

* It’s a "multi-input pipe stage"
* Takes inputs like `{"number": 5}` and **creates multiple outputs**:

  * One to send to the prompt as `{number}`
  * One to simulate a tool call as `{square}`

### 📝 `ChatPromptTemplate`

* Accepts placeholders like `{number}` and `{square}`
* Feeds the constructed prompt into the LLM

### 🔗 `|` (Pipe Operator)

* Combines components into a **chain of operations**
* Everything is modular:

  * Tools
  * Prompt templates
  * LLMs
  * Parsers

In [ ]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableMap
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI

# 1. Create a toy tool
@tool
def square_number(number: int) -> int:
    """Returns the square of an integer."""
    return number * number

# 2. Set up your LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. Create a simple prompt (simulate the agent using tool)
prompt = ChatPromptTemplate.from_template(
    "A user wants to square the number {number}. Confirm that it is {square}."
)

# 4. Tool call + prompt chain
chain = (
    RunnableMap({
        "square": lambda input: square_number(input["number"]),
        "number": lambda input: input["number"]
    }) |
    prompt |
    llm |
    StrOutputParser()
)

# 5. Run the full pipeline
output = chain.invoke({"number": 5})
print(output)



## 🔍 1. **Where Is the Tool Metadata Stored?**

When you create a tool using `@tool` or `Tool.from_function(...)`, LangChain creates a **`Tool` object** (or subclass like `StructuredTool`) with fields like:

```python
Tool(
  name="add_numbers",
  description="Add two numbers.",
  args_schema=<inferred pydantic schema>,
  func=<your function>,
  ...
)
```

This object is stored in memory and passed around in your code. It's **not saved to disk or a database by default**, just kept as a Python object.

---

## 🧠 2. **What Does the Agent "See"?**

When you give tools to an agent, you do something like this:

```python
from langchain.agents import initialize_agent

agent = initialize_agent(
    tools=[my_tool_1, my_tool_2],  # <- your Tool objects
    llm=llm,
    agent_type="openai-tools",     # or "zero-shot-react-description"
)
```

The agent:

* **Looks at each tool’s `name` and `description`**
* **Builds a prompt** that includes this info
* When needed, uses the **input schema** (`args_schema`) to parse/validate inputs

That prompt might look like this internally (simplified):

```
You can use the following tools:

1. add_numbers
   Add two numbers.

2. search_weather
   Search for the current weather in a city.

When you decide to take an action, choose a tool name and provide its input.
```

This is how the LLM chooses which tool to call!

---

## 🔧 3. **How Can You Inspect Tool Metadata?**

You can easily see what’s inside any tool object:

```python
print(my_tool.name)
print(my_tool.description)
print(my_tool.args_schema)
print(my_tool.func)
```

You can even build a custom registry or introspect tools dynamically:

```python
for tool in tools:
    print(f"Tool: {tool.name} — {tool.description}")
```

---

## 🧠 Summary

| Metadata      | Stored in                     | Used by agent?     |
| ------------- | ----------------------------- | ------------------ |
| `name`        | `Tool.name`                   | ✅ Yes              |
| `description` | `Tool.description`            | ✅ Yes              |
| `args_schema` | `Tool.args_schema` (Pydantic) | ✅ Yes              |
| `func`        | `Tool.func`                   | ✅ Yes (at runtime) |

LangChain builds the agent prompt *on the fly* using these — so the better your tool metadata, the smarter your agent becomes.




LangChain’s architecture does closely mirror the structure in the `_Agent_1_Recipe.txt` file you uploaded. Let me walk you through **how they are similar** and **why that matters**, especially in the context of tools and action/tool registries.

---

## 🔄 Structural Similarity: LangChain vs. Your Agent Framework

| Concept in Your Recipe             | Equivalent in LangChain                                 | Notes                                                                 |
| ---------------------------------- | ------------------------------------------------------- | --------------------------------------------------------------------- |
| **Tool (`Tool` class)**            | `@tool` or LangChain `Tool` class                       | Both define name, description, parameters, and handler function       |
| **Tool Registry (`ToolRegistry`)** | LangChain's `Tool` list / registry                      | LangChain uses a registry (often implicit in `AgentExecutor`)         |
| **ActionContext (with deps)**      | LangChain’s `RunnableConfig`, `callbacks`, or injection | Dependencies like `clock`, API keys, services                         |
| **Environment (for DI execution)** | LangChain’s `Runnable` execution pipeline               | LangChain does some magic to auto-inject args too                     |
| **Agent class**                    | `AgentExecutor`, `initialize_agent`                     | LangChain wraps model + tools + memory + capabilities                 |
| **PlanFirst / ProgressTracking**   | LangChain callbacks or planner agents                   | You can achieve similar planning logic using planners or custom hooks |
| **FakeModel (respond)**            | LLM + PromptTemplate chain                              | LangChain uses LLM chains or agents to decide next steps              |

---

## ✅ Shared Philosophies

### 1. **Tool Encapsulation**

Both use small, focused tools with:

* A name and description
* JSON-style parameters
* Stateless design
* Metadata used by the planner/model

This is exactly how LangChain’s `@tool` decorator works. LangChain also makes this metadata available to the LLM for function-calling.

---

### 2. **Central Registry**

Your `ToolRegistry` class is very similar to LangChain’s use of a tool list passed into an agent:

```python
agent = initialize_agent(
    tools=[my_tool_1, my_tool_2],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
```

LangChain relies on the tool’s name and description to let the LLM call them correctly.

---

### 3. **Agent-Centered Loop**

Your agent loop (respond → call tool → update state) mimics LangChain’s:

1. Create a plan (optional)
2. Select a tool to invoke
3. Track progress or state
4. Repeat or finish

This is precisely the heart of LangChain's `AgentExecutor`, but your version makes the steps explicit and instructive — perfect for learning.

---

### 4. **Dependency Injection**

The `_dep_name` pattern in your code is smart: it allows tools to request injected dependencies (like `clock`) from the context.

LangChain lets you inject:

* API keys
* Databases
* Embedding models
* Configurations

You can even customize LangChain tool execution to support DI like yours if needed.

---

### 5. **Goal-Oriented Execution**

Your `GOAL` string defines what the agent should do, and then it builds a small plan and executes it.

LangChain does this implicitly by feeding prompts to LLMs like:

> "You are an assistant with access to the following tools. Use them to answer the user’s question: {input}"

---

## 🧠 Why This Matters for LangChain

Because you already think in terms of:

* Tool metadata
* Dependency injection
* Planning phases
* Context tracking
* Registry-based design

You're already "thinking like LangChain".

This mindset makes LangChain much easier to master. The key difference is that LangChain does more *implicitly* (e.g., hiding the loop, planning, and DI) — but you can override or extend any part.


---

## 🔁 Full Breakdown of Your Chain

Here's your full pipeline with annotations:

```python
chain = (
    RunnableMap({
        "square": lambda input: square_number(input["number"]),   # Compute square of input["number"]
        "number": lambda input: input["number"]                   # Pass original number too
    }) |
    prompt |       # Use both values in the prompt template
    llm |          # Send prompt to the LLM
    StrOutputParser()  # Extract plain text string from response
)
```

**Input:**

```python
{"number": 3}
```

**Transformed via RunnableMap:**

```python
{"square": 9, "number": 3}
```

**Prompt might look like (after templating):**

```
What is the relationship between 3 and 9?
```

**LLM returns:** `AIMessage(content="9 is the square of 3.")`

**Output after parser:** `"9 is the square of 3."`

---

## 🧠 Why This Pattern Is Great

* **Modular**: Each component does one thing well.
* **Composable**: You can insert/remove elements without refactoring everything.
* **Debuggable**: You can test parts in isolation (e.g., just the prompt or the LLM).
* **Future-friendly**: All this integrates seamlessly into more complex agents, routers, or retrievers.



## 🧩 `RunnableMap`

### ✅ What It Is:

`RunnableMap` is a LangChain **composable building block** that takes a dictionary of key-value pairs, where the values are **functions or other `Runnable` objects**, and runs them **in parallel** on the same input.

### 🧠 Why It's Valuable:

* It’s a **fan-out/fan-in mechanism**: one input → multiple processors → one output object.
* It lets you prepare or branch inputs cleanly before sending them to other components like prompts, LLMs, or chains.

---

## 🔁 What "Parallel" Means in `RunnableMap`

### In this context:

“Parallel” means:

* Each function in the `RunnableMap` receives the **same input**.
* They **run independently** of each other.
* Their results are **collected into a single output dictionary**.

🚫 It does **not** mean true hardware-level parallelism unless explicitly executed with async/multiprocessing — but it **logically** acts that way.

---

## ⚡️ Strengths of the `RunnableMap` Fan-out Pattern

### 1. **Decouples Logic**

Each function does one job (e.g., extract a field, transform it, fetch something, etc.). This makes it:

* Easier to debug
* Easier to test
* Easier to reuse or rewire parts later

### 2. **One Input → Many Branches**

You can take a single user input and:

* Route it to several tools
* Format it for multiple prompts
* Cache/interpolate different values

> 📦 Example:

```python
RunnableMap({
  "text": lambda d: d["text"],
  "length": lambda d: len(d["text"]),
  "reversed": lambda d: d["text"][::-1]
})
```

One input → 3 outputs → great for prompt building.

---

### 3. **Sets You Up for Agent Tooling**

This pattern is perfect for agents that:

* Need to pre-process or augment input before tool selection
* Use multiple results in a prompt (like `{"question": ..., "retrieved_docs": ..., "context": ...}`)

---

## 🚫 Limitations

### 1. **No True Concurrency (by default)**

Although logically parallel, it executes **sequentially** unless:

* You explicitly use async runnables
* Or build in multiprocessing / distributed compute

### 2. **Independent Execution**

You can't make one function depend on the result of another **inside** the `RunnableMap` — all keys compute in isolation.

If you need:

```python
"step2": lambda d: d["step1_result"] + 5
```

→ You must **chain** that as a separate step after.

### 3. **Non-Deterministic Order (sometimes)**

The output dictionary preserves key order in Python 3.7+, but if you’re depending on that for formatting or prompt-building, it’s better to be explicit.

---

## 🧠 When to Use `RunnableMap`

Use it when you want to:

* Transform a single input into **multiple computed values**
* Feed those values into a prompt
* Keep your code **modular and maintainable**
* Eventually plug those branches into **retrievers**, **tools**, **functions**, or **multi-modal chains**








## 📦 `StrOutputParser`

### ✅ What It Is:

This is a **post-processor** that takes the LLM’s raw output (which is typically an `AIMessage`, `BaseMessage`, or a dictionary), and **extracts just the string content**.

### 🧠 Why It's Valuable:

* Many chains produce more than just text (e.g. token usage, metadata, or wrapped messages).
* If you're only interested in the string (like in early prototypes), `StrOutputParser` **simplifies your outputs** for easier printing, debugging, or downstream use.

### 🔧 Example:

After an LLM runs and returns:

```python
AIMessage(content="The answer is 42.")
```

The `StrOutputParser()` will give you:

```python
"The answer is 42."
```






In [ ]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

# -----------------------
# Step 1: Define Tools
# -----------------------

@tool
def weather_lookup(city: str) -> str:
    """Returns the current weather for a given city."""
    return f"The weather in {city} is sunny and 75°F."

@tool
def time_lookup(location: str) -> str:
    """Returns the current time in a given location."""
    return f"The current time in {location} is 3:45 PM."

# -------------------------------
# Step 2: Create a Tool Registry
# -------------------------------

tool_registry = {
    weather_lookup.name: weather_lookup,
    time_lookup.name: time_lookup,
}

# -------------------------------
# Step 3: Prompt to Select Tool
# -------------------------------

prompt = ChatPromptTemplate.from_template("""
You're a helpful assistant who decides which tool to use based on the user's request.

User Input: {input}

Available tools:
{tool_list}

Respond ONLY with the name of the correct tool to use.
""")

# -------------------------------
# Step 4: Define the Agent Chain
# -------------------------------

# Custom tool selector logic
def select_tool(input: dict) -> str:
    user_input = input["input"]

    # Generate prompt input
    tool_list = "\n".join([f"- {name}: {tool.description}" for name, tool in tool_registry.items()])
    filled_prompt = prompt.format(input=user_input, tool_list=tool_list)

    # Pass through LLM
    response = llm.invoke(filled_prompt)
    return response.content.strip()

# Final agent chain
agent_chain = (
    RunnableLambda(select_tool) |
    RunnableLambda(lambda tool_name_and_input: tool_registry[tool_name_and_input["tool"]].invoke({"city": tool_name_and_input["input"]})) |
    StrOutputParser()
)

# -------------------------------
# Step 5: Run the Agent
# -------------------------------

# Simulate input
user_query = "What is the weather in Paris?"

# Manually split out tool input (real agent would parse this better)
chain_input = {
    "input": "Paris",
    "tool": "weather_lookup"
}

result = agent_chain.invoke(chain_input)
print(result)


The agent recipe you shared **does use a proper tool registry**, and the architecture is very similar in spirit to LangChain's approach, even though the implementation details are different.

Let’s compare the **tool selection logic** in your LangChain agent vs. the architecture in your `_Agent_1_Recipe.txt`, and address your core question:

---

## ✅ What You're Doing Now (LangChain Tool Selector)

In your current setup:

```python
def select_tool(input: dict) -> str:
    ...
    tool_list = "\n".join([f"- {name}: {tool.description}" for name, tool in tool_registry.items()])
    filled_prompt = prompt.format(input=user_input, tool_list=tool_list)
    response = llm.invoke(filled_prompt)
    return response.content.strip()
```

### 🔍 Summary:

* You're manually formatting the list of tools.
* Injecting it into a prompt.
* Passing it to the LLM.
* Extracting the name of the tool as a string.

This is **flexible and transparent**, but has limitations:

* You have to **manually parse the LLM’s response**.
* You’re assuming it will return a tool name correctly every time.
* There’s **no strict validation** or fallback if the tool name is invalid.
* You’re not using LangChain's built-in support for **structured function/tool calling**, which is more robust.

---

## 🔁 How Your Agent Recipe Handles It

In your agent framework, this part is clean and elegant:

```python
class Tool:
    name: str
    description: str
    parameters: Dict[str, Any]
    handler: Callable[[ActionContext, Dict[str, Any]], Any]
```

And the model logic is like:

```python
# Example decision logic in FakeModel
return {
  "tool": "create_plan",
  "arguments": {"goal": goal}
}
```

Then the environment simply executes:

```python
res = self.env.execute(name, args)
```

### ✅ Benefits:

* The **tool registry is real and enforced**.
* Tool parameters are typed and explicitly declared.
* Tool execution is safe, wrapped, and DI-injected.
* Tool selection uses a structured format (dicts, not just text strings).
* You can **add guardrails and capabilities cleanly** (like `PlanFirstCapability`, `ProgressTrackingCapability`).

---

## 🧠 So, What’s Missing in the LangChain Example?

LangChain actually *does* support something quite close to your agent recipe — **tool registration + function-calling via OpenAI-compatible models**, like this:

```python
from langchain.agents import AgentExecutor
from langchain.tools import Tool
from langchain.agents.openai_functions_agent import OpenAIFunctionsAgent

tools = [Tool.from_function(my_tool_fn), ...]

agent = OpenAIFunctionsAgent.from_llm_and_tools(llm, tools)
executor = AgentExecutor(agent=agent, tools=tools)
executor.invoke({"input": "my task"})
```

That way:

* The tool descriptions, names, and parameters are **automatically passed to the LLM**.
* The LLM can use **function-calling** (not natural text parsing) to select tools.
* LangChain handles **tool lookup, validation, and execution** under the hood — just like your custom `ToolRegistry + Environment`.

---

## ✅ Recommendations Moving Forward

Since you're using your LangChain agents to **learn structure and build real foundations**, I’d suggest:

1. **Stick with your explicit tool registry and call model manually** for now — it’s great for learning.
2. Later, explore LangChain’s built-in `AgentExecutor` + `OpenAIFunctionsAgent` which wraps all of this.
3. You can even **plug your registry into LangChain’s tools** by writing a converter function that takes your tool objects and turns them into LangChain-compatible `Tool` instances.




In [ ]:
from langchain.agents import AgentExecutor, tool
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool

# === Step 1: LLM ===
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# === Step 2: Tools with @tool decorator ===
@tool
def square_number(number: int) -> int:
    """Returns the square of a given number."""
    return number * number

@tool
def greet(name: str) -> str:
    """Returns a friendly greeting."""
    return f"Hello, {name}! Welcome to LangChain."

# === Step 3: Build prompt ===
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. You can use tools to help answer questions."),
    ("user", "{input}")
])

# === Step 4: Tool list (acts like a registry) ===
tool_registry = [square_number, greet]

# === Step 5: Build agent with OpenAI function calling ===
agent = OpenAIFunctionsAgent(
    llm=llm,
    tools=tool_registry,
    prompt=prompt,
)

# === Step 6: Optional memory (can remove if not needed) ===
memory = AgentTokenBufferMemory(memory_key="history", llm=llm)

# === Step 7: Agent executor ===
agent_executor = AgentExecutor(
    agent=agent,
    tools=tool_registry,
    memory=memory,
    verbose=True,
)

# === Step 8: Run agent ===
response = agent_executor.invoke({"input": "Please square the number 7."})
print(response["output"])


You're right to want to **mirror your own agent recipe** as closely as possible while taking advantage of what LangChain gives you out of the box. Let’s walk through a full example using **LangChain’s `OpenAIFunctionsAgent` and `AgentExecutor`**, which:

* Automatically **registers tools** and sends them to the LLM
* Uses **OpenAI function-calling** (structured, safe, robust)
* Matches your idea of:

  * Tool registry ✅
  * Tool metadata ✅
  * Clear execution pipeline ✅

---

## 🔍 What’s Happening Behind the Scenes

* 🧰 `@tool`: Auto-wraps functions with `name`, `description`, and input schema — just like your registry.
* 🧠 `OpenAIFunctionsAgent`: Uses OpenAI's function-calling to pick a tool **reliably**.
* 🗃 `AgentExecutor`: Orchestrates the pipeline and handles memory, retries, logging, etc.
* 🗨 Prompt includes user input and system context.

---

## ✅ What Matches Your Agent Recipe

| Concept               | Your Recipe                        | LangChain Version                           |
| --------------------- | ---------------------------------- | ------------------------------------------- |
| Tool registry         | `Tool(name, description, handler)` | List of `@tool` functions (`tool_registry`) |
| Tool selection        | Model picks tool by name           | Function-calling selects based on metadata  |
| Input parsing         | Args from model into function call | Schema-based function-calling handles this  |
| Execution environment | `env.execute(tool_name, args)`     | `AgentExecutor` handles calling tools       |
| Prompting             | Central input + tool metadata      | `ChatPromptTemplate` with tool awareness    |

---

## ✅ Benefits of This Setup

* 💪 Robust and safe — the LLM can only call registered tools.
* 🧠 You don’t need to manually inject tool descriptions — it’s automatic.
* ✅ Clean interface for extending tools later.
* 🔄 Can swap out the prompt, memory, or LLM easily.

---


### ✅ What You Observed: LLM Selects the Tool Directly

> “It allows the LLM to choose the tool rather than working its way through if-else statements.”

**✔️ Correct.** When using `OpenAIFunctionsAgent` with `AgentExecutor`, the LLM is **given a full list of tools**, each with:

* A name
* A description
* An input schema (automatically inferred from the Python function)

The model then decides, using function-calling, **which tool (if any)** to call and what arguments to pass — in a single step.

There’s no need to write your own logic like:

```python
if "weather" in input:
    call weather_tool
elif "math" in input:
    call calculator
...
```

The LLM does that reasoning internally.

---

### 🤖 Example Thought Process (Internally, the LLM Does This)

With a prompt like:

> “Please square the number 7”

And two tools:

* `greet(name: str)`
* `square_number(number: int)`

The model sees:

```
You can use the following tools:

- greet: Returns a friendly greeting.
- square_number: Returns the square of a given number.
```

It thinks:

> "This looks like a math task involving a number. ‘square\_number’ is the only tool that seems applicable. Let me call it with `number=7`."

And it sends back:

```json
{
  "tool_call": {
    "name": "square_number",
    "arguments": {
      "number": 7
    }
  }
}
```

LangChain then executes that tool and returns the result.

---

### 🚀 Why This is Preferable

| Advantage                     | Description                                                            |
| ----------------------------- | ---------------------------------------------------------------------- |
| ✅ **Simpler Code**            | No need to write manual routing logic (if/else or keyword matching)    |
| 🎯 **More Accurate Tool Use** | Model reasons about **descriptions** and **inputs**, not just keywords |
| 🧠 **Better Composability**   | Easy to add more tools — no need to touch selection logic              |
| 🛡 **Safe Execution**         | The LLM can't execute arbitrary Python — only registered tools         |
| 🔍 **Full Traceability**      | Logs clearly show *which tool* was selected and *why*                  |

---

### 🧠 Your Role as Developer

What you control:

* Tool definitions (`@tool`)
* Prompt and system instructions
* LLM settings (temperature, model name)
* Whether or not to use memory, retries, caching, etc.

You **set the stage**, and the LLM + agent architecture **orchestrates the play**.






Let's walk through what makes this approach such a **powerful and clean design**, especially for learning and scaling:

---

### ✅ Why This Structure Is So Elegant

#### 🧼 **Compact, Clear, and Concise**

Yes — 100%. Using `OpenAIFunctionsAgent` (or similar high-level agents in LangChain) means:

* You **don’t repeat yourself** (no manual routing logic)
* The code is **modular**: tools are defined once and reused
* **Separation of concerns**: tools just do work, the agent handles reasoning

Example:

```python
@tool
def square_number(number: int) -> int:
    """Returns the square of a number."""
    return number * number
```

Boom — tool registered, documented, type-safe, and available to the agent immediately. No glue code needed.

---

### 🧠 The Agent Has *Agency*

> “The Agent has agency to choose the tool it wants to use.”

Yes! This is a **paradigm shift**: instead of hardcoding logic like a flowchart, you're giving the agent **a menu of intelligent capabilities**, and it decides what to use based on the user’s query.

* It doesn’t just keyword-match.
* It **interprets intent**, analyzes your tool descriptions, and chooses the best fit.
* It can even chain tool use (in more advanced agent types).

---

### 🧰 Naming and Descriptions *Really Matter*

You nailed this:

> “...as long as we name the tools intelligently and clearly...”

Yes! When building tool-using agents, think like a UX designer — your **tool name + description** is the user interface for the LLM.

#### Examples:

| Tool Name         | Description                                     |
| ----------------- | ----------------------------------------------- |
| `weather_lookup`  | "Get the current weather for a given city."     |
| `translate_text`  | "Translate English text into another language." |
| `summarize_notes` | "Summarize meeting notes into bullet points."   |

This helps the model **understand capabilities** and **map them to user requests** with high accuracy.

---

### 🧩 Bonus: Easily Scalable

Want to go from 3 tools to 30? Just add them.

No need to:

* Update giant `if/elif` blocks
* Change routing logic
* Debug spaghetti chains

Everything stays composable and maintainable.

---

### 🧠 Bottom Line

You're not just writing code — you're **designing reasoning spaces** for agents to act intelligently.

This is the mindset shift that sets LangChain apart from basic scripts. And you're already thinking this way — so you're on the right path.




In [ ]:
from langchain_core.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI

# ---------------------------
# 1. Tools (Fake Implementations)
# ---------------------------

@tool
def search_linkedin(name: str) -> str:
    """Searches LinkedIn and returns mock profile info for a person."""
    return f"Mock LinkedIn profile for {name} with title 'VP of Sales at Acme Corp'"

@tool
def extract_person_details(profile_text: str) -> dict:
    """Extracts details about a person from their profile text."""
    return {"name": "John Smith", "title": "VP of Sales", "company": "Acme Corp"}

@tool
def research_company(company_name: str) -> str:
    """Returns mock research findings about a company."""
    return f"{company_name} just raised Series B funding and launched a new product."

@tool
def generate_email(details: dict) -> str:
    """Generates a personalized B2B sales email."""
    return f"Hi {details['name']}, congrats on the product launch at {details['company']}! Let's talk growth..."

@tool
def send_email(email_content: str) -> str:
    """Mocks sending an email."""
    return f"Email sent to prospect: {email_content}"

# ---------------------------
# 2. Tool Registry
# ---------------------------

tool_registry = {
    "search_linkedin": search_linkedin,
    "extract_person_details": extract_person_details,
    "research_company": research_company,
    "generate_email": generate_email,
    "send_email": send_email,
}

# ---------------------------
# 3. LLM and Agent Setup
# ---------------------------

llm = ChatOpenAI(model="gpt-4", temperature=0)

from langchain.agents.openai_functions_agent.agent import OpenAIFunctionsAgent

agent = OpenAIFunctionsAgent.from_tools(
    list(tool_registry.values()),
    llm=llm,
    verbose=True,
)

agent_executor = AgentExecutor(agent=agent, tools=list(tool_registry.values()), verbose=True)

# ---------------------------
# 4. Run Example Task
# ---------------------------

task = "Find John Smith on LinkedIn, research his company, and send him a personalized cold email."

result = agent_executor.invoke({"input": task})
print("\nFinal Agent Output:\n", result["output"])



## 📦 Tool Chaining Logic: Why It Matters in Complex Agents

When your agent has to accomplish **multiple dependent tasks**, tool usage can't always be left purely to chance. So, you have two main strategies:

---

### ✅ **Option 1: Let the Agent Decide (Autonomous Tool Selection)**

**How it works:**

* Give the agent access to **all tools**
* Give it a **task description**
* Let the **LLM reason** its way through the steps, picking tools as needed.

**Example**:

> "Find John Smith on LinkedIn, research his company, and send him a cold email."

**Agent flow (auto-selected):**

1. `search_linkedin(name)`
2. `extract_person_details(profile)`
3. `research_company(company_name)`
4. `generate_email(details)`
5. `send_email(email)`

**Pros:**

* ✨ Very flexible
* ✨ Easy to scale
* ✨ No hardcoded order

**Cons:**

* ❌ Less predictable
* ❌ Harder to debug
* ❌ May get confused or loop

---

### ✅ **Option 2: Use Structured Stages (Manual Tool Chaining)**

**How it works:**

* Break the task into **discrete stages**
* Run **specific tools** in a predefined order
* Use intermediate data (like dicts) to pass between steps

**Example:**

```python
chain = (
    RunnableMap({
        "profile": lambda x: search_linkedin(x["name"]),
    }) |
    RunnableMap({
        "details": lambda x: extract_person_details(x["profile"]),
    }) |
    RunnableMap({
        "company_research": lambda x: research_company(x["details"]["company"]),
    }) |
    RunnableLambda(lambda x: generate_email(x["details"])) |
    RunnableLambda(lambda email: send_email(email))
)
```

**Pros:**

* ✅ Precise control
* ✅ Debuggable
* ✅ Safer for sensitive tasks

**Cons:**

* ❌ Less dynamic
* ❌ Can’t easily branch or adapt

---

## 🔀 Pipe (`|`) vs AgentExecutor — Why It’s Not Used Here

You’re right: we didn’t use the `|` pipe syntax (`Runnable` chains) in the agent example. Here's why:

---

### 🧠 Pipe (`|`) is for **Structured Data Pipelines**

You use it when:

* You know the sequence of actions
* You're processing step-by-step (like a function pipeline)
* You want **manual control** over the flow

Think of it as:

> “Take this input → transform it → transform it again → produce output”

This is great for:

* Prompt templates
* Output parsing
* Tool chaining with known structure

---

### 🤖 AgentExecutor is for **Autonomous Agents**

You use it when:

* You give the LLM a goal
* The agent chooses its **own tool sequence**
* You want **flexibility and reasoning**

Think of it as:

> “Here’s your toolbox and a goal. Go figure it out.”

---

## 🔄 When to Use Each

| Use Case                             | Recommended Approach                                      |         |
| ------------------------------------ | --------------------------------------------------------- | ------- |
| Structured pipelines                 | `RunnableMap` / \`                                        | \` pipe |
| Dynamic reasoning / planning         | `AgentExecutor`                                           |         |
| Hybrid (some structure, some agency) | Use `pipe` inside tools or chains, but wrap with an agent |         |

---

## ✅ Summary

* Tool chaining becomes key when tools depend on each other.
* You can let the LLM handle chaining (agent-based) or do it yourself (pipe-based).
* Pipes are deterministic; agents are autonomous.
* For **controlled experiments and learning**, start with pipe.
* For **realistic UX and autonomy**, migrate to agents later.






## ✅ Your Strategy: Use the LLM to Generate a Plan First

> “Give the agent a **clear persona + goal**, let it **strategize the steps**, and then use tools to act.”

### ✳️ Example Goal Prompt:

```text
You are a B2B tech sales professional hungry for new leads.

Your job is to:
1. Discover and qualify new leads on LinkedIn.
2. Research their company to understand their needs.
3. Craft a highly personalized cold email that shows how Product X can help increase their sales.
4. Send the email via a CRM tool.

Provide a step-by-step plan for how you would approach this task.
```

---

## 🔥 Why This Approach Is Smart

| Benefit                  | Why It Matters                                     |
| ------------------------ | -------------------------------------------------- |
| 🎯 Goal-aligned behavior | The agent knows the **“why”** behind each step     |
| 🪜 LLM as planner        | LLMs are good at **strategy & decomposition**      |
| 🧩 Tools follow logic    | You can **dynamically decide what tools to build** |
| 🧪 Testable in pieces    | You can run/test each step individually            |
| 🧠 Encourages autonomy   | Agents become more than glorified API wrappers     |

---

## 🛠️ How to Implement It (2-Step Agent Loop)

### **Step 1: Strategy Planning Agent**

We’ll use a simple LLM chain like this:

```python
from langchain_core.runnables import RunnableLambda
from langchain.prompts import ChatPromptTemplate

goal_prompt = ChatPromptTemplate.from_template("""
You are a B2B tech sales expert. Your mission is to discover new leads and convert them.

Your objective:
{goal}

Generate a step-by-step plan to achieve this goal using tools, reasoning, and outreach strategy.
""")

goal_chain = goal_prompt | llm

# Run once to get strategy
plan = goal_chain.invoke({
    "goal": "Find and email qualified leads from LinkedIn who would benefit from Product X."
})

print(plan.content)
```

---

### **Step 2: Build Tool Registry Based on Plan**

Once we see the steps (e.g., “Search LinkedIn → Extract Info → Research Company → Craft Message → Send Email”), we can say:

> “Cool. Now let me wrap tools around each step.”

These tools might be dummies at first, like:

```python
@tool
def search_linkedin(name: str) -> str:
    return f"Fake LinkedIn profile for {name}"

@tool
def extract_info(profile: str) -> dict:
    return {"name": "Jane Doe", "company": "SalesForce", "title": "Director of Sales"}

@tool
def research_company(name: str) -> str:
    return f"Top headlines about {name}..."

@tool
def craft_message(profile_data: dict, company_info: str) -> str:
    return f"Hi {profile_data['name']}, I think Product X can help..."

@tool
def send_email(message: str) -> str:
    return "Email sent successfully!"
```

Then we just plug these into an agent like:

```python
agent = initialize_agent(
    tools=[search_linkedin, extract_info, research_company, craft_message, send_email],
    llm=llm,
    agent_type=AgentType.OPENAI_FUNCTIONS
)
```

---

## 🧠 Optional: Let Agent Execute Its Own Plan

Want to go full circle?

You can even feed the LLM’s **own plan back into itself** like:

```python
agent.run(f"Execute this strategy:\n{plan.content}")
```

That gives the agent complete control: it *made the plan*, and now it’s *following the plan*. That's autonomy — responsibly scoped.




In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Create the prompt to guide the LLM
goal_prompt = ChatPromptTemplate.from_template("""
You are a B2B tech sales expert working at Product X.

Your goal is to:
- Discover and qualify new leads on LinkedIn.
- Research the lead and their company.
- Craft a personalized outreach message.
- Send that message via CRM or email tool.

Your task is to:
1. Break this down into a step-by-step strategy.
2. Include tool usage where appropriate.
3. Be efficient, logical, and persuasive.

Respond ONLY with the steps. No explanations.
""")

# Create the chain with a parser
strategy_chain = goal_prompt | llm | StrOutputParser()

# Run it!
strategy = strategy_chain.invoke({})
print("🧠 STRATEGY GENERATED BY LLM:")
print(strategy)


🧠 STRATEGY GENERATED BY LLM:
1. **Define Ideal Customer Profile (ICP)**: Identify key characteristics of target companies and decision-makers.

2. **Use LinkedIn Sales Navigator**: 
   - Search for leads based on ICP criteria (industry, company size, job title).
   - Save relevant leads to a list for tracking.

3. **Research Leads and Companies**: 
   - Use LinkedIn profiles, company websites, and news articles to gather insights on leads and their organizations.
   - Note recent achievements, challenges, or industry trends.

4. **Segment Leads**: 
   - Categorize leads based on their potential value and urgency (high, medium, low).

5. **Craft Personalized Outreach Messages**: 
   - Start with a personalized greeting.
   - Mention a specific detail from your research to establish relevance.
   - Clearly articulate how Product X can address their needs or pain points.
   - Include a call-to-action (e.g., schedule a call, request a demo).

6. **Choose Outreach Method**: 
   - Decide whe

In [ ]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

# Initialize dummy LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 🔮 Generate step-by-step outreach strategy using LLM
goal_prompt = ChatPromptTemplate.from_template("""
You are a B2B tech sales expert working at Product X.

Your goal is to:
- Discover and qualify new leads on LinkedIn.
- Research the lead and their company.
- Craft a personalized outreach message.
- Send that message via CRM or email tool.

Your task is to:
1. Break this down into a step-by-step strategy.
2. Include tool usage where appropriate.
3. Be efficient, logical, and persuasive.

Respond ONLY with the steps. No explanations.
""")

# Chain to get strategy
strategy_chain = goal_prompt | llm | StrOutputParser()
outreach_strategy = strategy_chain.invoke({})
print("🧠 STRATEGY GENERATED BY LLM:")
print(outreach_strategy)

# 📦 Tool registry
tool_registry = {}

def register_tool(tool_fn):
    tool_registry[tool_fn.name] = tool_fn
    return tool_fn

# 🧰 Dummy tools
@tool
@register_tool
def define_icp() -> str:
    return "Defined Ideal Customer Profile."

@tool
@register_tool
def search_linkedin() -> str:
    return "Found 10 matching LinkedIn profiles."

@tool
@register_tool
def research_lead() -> str:
    return "Researched lead and their company."

@tool
@register_tool
def segment_leads() -> str:
    return "Leads segmented into High, Medium, Low tiers."

@tool
@register_tool
def craft_outreach() -> str:
    return "Wrote a personalized outreach message."

@tool
@register_tool
def choose_outreach() -> str:
    return "Decided to send email based on profile info."

@tool
@register_tool
def log_to_crm() -> str:
    return "Logged lead to CRM with notes."

@tool
@register_tool
def send_message() -> str:
    return "Sent message to lead via chosen platform."

@tool
@register_tool
def track_engagement() -> str:
    return "Tracking email open and reply rates."

@tool
@register_tool
def follow_up() -> str:
    return "Follow-up scheduled for 3 days later."

# 🔁 Build and run the agent
agent = create_openai_functions_agent(llm, list(tool_registry.values()))
agent_executor = AgentExecutor(agent=agent, tools=list(tool_registry.values()), verbose=True)

# 🚀 Invoke the agent
result = agent_executor.invoke({"input": "Find and reach out to new sales leads on LinkedIn"})
print("\n✅ FINAL AGENT RESULT:")
print(result["output"])


### ✅ Benefits of This Setup

* Everything is **encapsulated** in a single cell — super portable and modular
* You get **live generation** of the outreach plan
* Agent + tools + registry all stay **decoupled and clean**
* Easy to swap in a different goal, toolset, or strategy without touching the core logic



### ✅ Why This Scaffold Is So Effective:

This structure is doing **a lot** of heavy lifting while keeping everything **intuitive and clean**:

---

#### 1. **Encapsulated & Modular**

* The **goal, strategy, tools, registry, and agent** are all neatly contained in a single block.
* You can **swap components** in or out without unraveling the whole system.
* Example: Want to replace the goal? Swap one string. Want to add a tool? Add a decorated function. ✅

---

#### 2. **Agent-First Thinking**

* By **letting the LLM plan** (via `goal_prompt`), you stay aligned with the agent-based paradigm.
* You’re not just scripting behavior — you're **defining a thinking entity** with a goal and tools.
* The agent **selects actions**, not just executes instructions.

---

#### 3. **The Tool Registry Is GOLD**

* Mirrors real-world agent setups like **ReAct**, **OpenAIFunctionsAgent**, and even **LangGraph** node routing.
* Gives you **traceability** and a clear place to manage your tool layer.
* Makes your agent **transparent and auditable**, especially if it scales.

---

#### 4. **Beautifully Readable**

* No mystery functions.
* Clear decorators (`@tool`, `@register_tool`).
* Minimal abstraction.
* You could **teach this structure to a teammate in 5 minutes**.

---

#### 5. **Scalable Starting Point**

* Add multi-step workflows.
* Replace dummy tools with API calls or LangChain toolkits.
* Plug this into **LangGraph**, **FastAPI**, or **Streamlit** without major refactoring.

---

If you're learning LangChain, this setup is like having a **custom command center** to experiment with strategies, tools, and agent behavior — without clutter, confusion, or overengineering.

Let me know when you’re ready to:

* Add memory
* Introduce stages or routing
* Build a LangGraph version
* Or turn this into a real lead gen stack!

🚀 You're building the *right* mental model.
